# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from fasthtml.common import *
from monsterui.all import *
from memexplatform_obsidian.commons import config, MountPaths
from memexplatform.ui.structure import ifhtmx
from memexplatform_obsidian.mdmanager import ObsidianHTMLRenderer, get_obsidianmd_ast, get_subdirs
from mistletoe import Document
import urllib
from memexplatform_obsidian.jupyter import render_nb
from memexplatform_obsidian.mdmanager import get_title
from typing import Optional
import pathlib
import mimetypes

In [ ]:
# StreamingResponse??

In [ ]:
#| export
def create_app():
    hdrs = Theme.neutral.headers(highlightjs=True)
    Mermaid = MermaidJS()
    Katex = KatexMarkdownJS()
    D3 = Script(src="https://cdn.jsdelivr.net/npm/d3@7")
    PyscriptCSS=Link(rel="stylesheet", href="https://pyscript.net/releases/2025.3.1/core.css", type="text/css"),
    PyscriptJS=Script(src="https://pyscript.net/releases/2025.3.1/core.js", type='module')
    SurrealGlobalAdd = Script("surreal.globalsAdd());");
    Plotly = Script(src="https://cdn.plot.ly/plotly-3.0.1.min.js")
    add_hdrs = [Mermaid, Katex, D3, PyscriptCSS, PyscriptJS, SurrealGlobalAdd, Plotly]
    newhdrs = hdrs +add_hdrs
    mount_routes = []

    app = FastHTML(hdrs=newhdrs, live=True, exts='ws', routes=mount_routes)
    # app.add_middleware(SessionSyncMiddleware)
    rt = app.route
    return app, rt

app, rt = create_app()

In [ ]:
MountPaths.gradio_url.value

'/obsidian_gradio'

In [ ]:
#| export
@rt
def index(request:Request):

    # listteam_comp = Container(Div('Team'), hx_get=f'/portal/list_team', hx_target='#listteam', id='listteam', hx_swap='innerHTML',hx_vals=json.dumps({"folder": str(config.NBPATH)}),  hx_trigger='load')
    comp = Div("Obsidian Companion")
    
    return ifhtmx(request, comp)

In [ ]:
vault_path = config.OBSIDIAN_VAULT
file = "Clippings/Fuck%20You%2C%20Show%20Me%20The%20Prompt.%20%E2%80%93%202.md"
# file = "Fuck%20You%2C%20Show%20Me%20The%20Prompt.%20%E2%80%93%202.md"
search_term = urllib.parse.unquote(file); print(search_term)
file_path = vault_path / search_term; file_path
base = get_subdirs(vault_path)[0]; base
# next(base.rglob(search_term), None)
# list(base.rglob("*"))
# list(base.rglob("*"))
print(list(base.rglob("*")))
next(base.rglob(search_term), None)
file = "Clippings"

Clippings/Fuck You, Show Me The Prompt. – 2.md
[Path('/Users/rahul1.saraf/rahuketu/programming/notesobs/Clippings/Frequently Asked Questions (And Answers) About AI Evals – Hamel’s Blog.md'), Path('/Users/rahul1.saraf/rahuketu/programming/notesobs/Clippings/How I Built a Learning Machine.md'), Path('/Users/rahul1.saraf/rahuketu/programming/notesobs/Clippings/ContextCite Attributing Model Generation to Context.md'), Path('/Users/rahul1.saraf/rahuketu/programming/notesobs/Clippings/Apertus a fully open, transparent, multilingual language model.md'), Path('/Users/rahul1.saraf/rahuketu/programming/notesobs/Clippings/How to solve any problem? - supermemo.guru.md'), Path('/Users/rahul1.saraf/rahuketu/programming/notesobs/Clippings/From GPT-2 to gpt-oss Analyzing the Architectural Advances.md'), Path('/Users/rahul1.saraf/rahuketu/programming/notesobs/Clippings/Fuck You, Show Me The Prompt. – 2.md')]


In [ ]:
#| export
def resolve_note_path(vault:Path, file:str) -> Path | None:
    search_term = urllib.parse.unquote(file); search_term
    rel_path = Path(search_term); rel_path
    fname = rel_path.name
    fldr = rel_path.parent; fldr
    pfname = None
    extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.svg', '.webp', '.tiff', '.ico',
                    '.base', '.canvas', '.md', '.qmd', '.ipynb'
                )
    if fldr.name == "":
        if fname.lower().endswith(extensions):
            for sub in get_subdirs(vault):
                pfname = next(sub.rglob(fname), None)
                if pfname: break
        else:
            mdfname = fname +".md"
            for sub in get_subdirs(vault):
                pfname = next(sub.rglob(mdfname), None)
                if pfname: break
    else:
        if fname.lower().endswith(extensions):
            pfname = next((vault/fldr).rglob(fname), None)
        else:
            mdfname = fname +".md"
            pfname = next((vault/fldr).rglob(mdfname), None)
    if pfname: return pfname
    else: 
        lsfldr = vault/search_term
        if lsfldr.is_dir(): return lsfldr
        else: return None

In [ ]:
#| export
@rt
def open(request: Request, file: str = "", title: Optional[str]=None):
    vault_path = config.OBSIDIAN_VAULT
    file_path = resolve_note_path(vault_path, file)
    if file_path is None: return Div(f"Path not found: {urllib.parse.unquote(file)}")

    # If it's a directory → list contents
    if file_path.is_dir():
        items = []
        for p in sorted(file_path.iterdir()):
            rel_path = p.relative_to(vault_path)
            href = MountPaths.open.to(file=rel_path)
            items.append(Li(A(p.name, href=href)))
        return ifhtmx(request,
            Div(
                f"Listing for {file_path}:",
                Ul(*items)
            )
        )
    # If it's a file → show content (assuming text)
    if file_path.is_file():
        if file_path.name.lower().endswith(".ipynb"):
            return ifhtmx(
                    request,
                    Container(
                        H1(file_path.stem, cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                        render_nb(file_path)
                ))
        try:
            text = file_path.read_text(encoding="utf-8")
        except Exception:
            text = "[Unable to read file]"

        with ObsidianHTMLRenderer() as renderer:
            doc = get_obsidianmd_ast(text)
            html = renderer.render(doc)
            title = get_title(doc)
            

        return ifhtmx(
            request,
            Container(
                H1(title if title else file_path.stem, cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                Card(NotStr(apply_classes(html)))
            )
        )

In [ ]:
#| export
def iter_file(path, chunk_size=8192):
    with open(path, "rb") as f:
        while chunk := f.read(chunk_size):
            yield chunk

In [ ]:
fname = 'Test.m4a'
mime = mimetypes.guess_type(fname); mime[0]
FileResponse?

Init signature:
FileResponse(
    path: 'str | os.PathLike[str]',
    status_code: 'int' = 200,
    headers: 'typing.Mapping[str, str] | None' = None,
    media_type: 'str | None' = None,
    background: 'BackgroundTask | None' = None,
    filename: 'str | None' = None,
    stat_result: 'os.stat_result | None' = None,
    method: 'str | None' = None,
    content_disposition_type: 'str' = 'attachment',
) -> 'None'
Docstring:      <no docstring>
File:           /opt/homebrew/Caskroom/miniforge/base/envs/automlops/lib/python3.11/site-packages/starlette/responses.py
Type:           type
Subclasses:     

In [ ]:
#| export
def CollapsibleBlocks(title, comp):
    return Div(
            Input(type='checkbox'),
            Div(title, cls='collapse-title font-semibold'),
            Div(comp, cls='collapse-content text-sm'),
            cls='collapse collapse-plus border'
        )

        

In [ ]:
to_xml(CollapsibleBlocks('T', 'c'))

'<div class="collapse collapse-plus border">\n  <input type="checkbox" class="uk-input ">\n  <div class="collapse-title font-semibold">T</div>\n  <div class="collapse-content text-sm">c</div>\n</div>\n'

In [ ]:
#| export
@rt
def embed(request: Request, file: str = "", ext:Optional[str]=None, title:Optional[str]=None):
    VIDEO_EXTS = {".mp4", ".webm", ".ogg", ".mov", ".mkv"}
    AUDIO_EXTS = {".mp3", ".wav", ".ogg", ".m4a", ".flac"}
    IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp"}
    TEXTLIKE_EXTS = {".md", ".qmd", ".canvas", ".base"}
    vault_path = config.OBSIDIAN_VAULT
    file_path = resolve_note_path(vault_path, file)
    if file_path is None: return Div(f"Path not found: {urllib.parse.unquote(file)}")

    # If it's a directory → list contents
    if file_path.is_dir():
        items = []
        for p in sorted(file_path.iterdir()):
            rel_path = p.relative_to(vault_path)
            href = MountPaths.open.to(file=rel_path)
            items.append(Li(A(p.name, href=href)))
        return ifhtmx(request,
            Div(
                f"Listing for {file_path}:",
                Ul(*items)
            )
        )
    # If it's a file → show content (assuming text)
    if file_path.is_file():
        if ext in IMAGE_EXTS:
            headers = {'Content-Type': f"image/{ext[1:]}"}
            
            # Add alt and title to headers if available/derivable
            # 'title' is passed as a parameter to the embed function
            if title:
                headers['X-Image-Title'] = title
            
            # 'alt' is not passed directly to the embed function, so derive it from the file name
            # This 'alt' would typically be used by the HTML <img> tag that references this image.
            # Adding it as a custom header here provides it in the response, as per instruction.
            alt_text = pathlib.Path(file).stem
            headers['X-Image-Alt'] = alt_text
            
            return Response(file_path.read_bytes(), headers=headers)

        # --- Video files ---
        if ext in VIDEO_EXTS:
            # mime = f"video/{ext[1:]}" if ext else "video/mp4"
            mime = mimetypes.guess_type(file_path.name, strict=False)
            headers = {"Accept-Ranges": "bytes"}  # Important for seeking
            # return Response(file_path.read_bytes(), headers=headers)
            return FileResponse(file_path, media_type=mime[0], headers=headers)

        # --- Audio files ---
        if ext in AUDIO_EXTS:
            # mime = f"audio/{ext[1:]}" if ext else "audio/mpeg"
            mime = mimetypes.guess_type(file_path.name, strict=False)
            headers = {"Accept-Ranges": "bytes"}  # Important for seeking
            # return Response(file_path.read_bytes(), headers=headers)
            return FileResponse(file_path, media_type=mime[0], headers=headers)

        if file_path.name.lower().endswith(".ipynb"):
            return ifhtmx(
                    request,
                    Container(
                        H1(file_path.stem, cls='uk-h1 text-4xl font-bold mt-12 mb-6'),
                        render_nb(file_path)
                ))
        try:
            text = file_path.read_text(encoding="utf-8")
        except Exception:
            text = "[Unable to read file]"

        with ObsidianHTMLRenderer() as renderer:
            doc = get_obsidianmd_ast(text)
            html = renderer.render(doc)
            title = get_title(doc)

        # return Response(html, media_type="text/html")
        return CollapsibleBlocks(
           Div(title if title else file_path.stem), 
           NotStr(apply_classes(html))
        )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()